In [5]:
import pandas as pd

def map_chunk(df_chunk):
    total_playtime = 0
    count = 0
    for recommended, playtime in zip(df_chunk["recommended"], df_chunk["author.playtime_forever"]):
        if not recommended:
            count += 1
            total_playtime += playtime
    return {"total_playtime": total_playtime, "count": count}

def shuffle(mapped_results):
    total_playtime = 0
    total_count = 0
    for r in mapped_results:
        total_playtime += r["total_playtime"]
        total_count += r["count"]
    return total_playtime, total_count

def reduce(total_playtime, total_count):
    if total_count == 0:
        return 0
    # Переводим минуты в часы
    return total_playtime / total_count / 60

def mapReduce(file_path, chunksize=200_000):
    mapped = []
    for chunk in pd.read_csv(
        file_path,
        usecols=["recommended", "author.playtime_forever"],
        chunksize=chunksize,
        low_memory=True,
        engine="c"
    ):
        chunk["recommended"] = chunk["recommended"].astype(bool)
        chunk["author.playtime_forever"] = pd.to_numeric(chunk["author.playtime_forever"], errors="coerce").fillna(0)
        mapped.append(map_chunk(chunk))

    total_playtime, total_count = shuffle(mapped)
    average_playtime_hours = reduce(total_playtime, total_count)
    return total_count, average_playtime_hours

def main():
    file_path = r"C:\Users\Firo\PycharmProjects\InstituteProject-course--4-\BigData\lab1\steam_reviews.csv"
    total_count, average_playtime_hours = mapReduce(file_path)
    print("Количество пользователей, которые НЕ рекомендуют игру:", total_count)
    print("Среднее время игры (часы):", average_playtime_hours)

if __name__ == "__main__":
    main()


Количество пользователей, которые НЕ рекомендуют игру: 2724761
Среднее время игры (часы): 330.4629501449852
